In [ ]:
import SILIA
import numpy as np
from scipy.signal import square
import plotly.express as px #I like using plotly for my graphs, but feel free to use matplotlib or any other library
import plotly.graph_objects as go
from tqdm import tqdm
from matplotlib import animation, rc
import matplotlib.pyplot as plt

In this tutorial, we will go over using SILIA for higher dimensional data. Please see the basic tutorial prior to moving on to this one. If you haven't done so already, all depencencies can be installed using pip, we use plotly and pillow which do not generally come pre-installed. In this case, we will simulate fluoresence microscopy data where the sample fluoresces at 100Hz and the microscope takes images on a 100x100 pixel array. The fluoresence is concentrated in a few randomly placed circles with a signal peak of 2 while the background has a peak of 0.2, resulting in a 10:1 ratio for the concentration of the fluorescent molecule between the circles and the background. We will see if the Lock-in can recover this result despite significant noise. 

First, let us create our time axis and note down our frequency

In [ ]:
sampling_rate = 5000 #Hz
sim_time = 1 #second
#Time axis
time = np.arange(0, sim_time, 1/sampling_rate)
#Frequency (Hz)
freq = 100
#Number of rows in each frame
rows = 100
#Number of columns in each frame
cols = 100

Next, let's simulate our 100Hz reference signal

In [ ]:
references = [{'time' : time, 'signal' : square(2 * np.pi * freq * time)}]

Now, let's generate our data. Our signal is represented by a dictionary which contains the time axis under the key, 'time'. The actual signal data is under the key 'signal' and is a 3D array in this case. The first index of this array represents the time axis, i.e. signal['signal'][i] would refer to the image frame at the i'th timestep. The next two indices represent the coordinates of an individual pixel. i.e. signal['signal'][:, x, y] is a 1D array which represents the signal value over time for a single pixel. 

In [ ]:
def inside(centers, radii, point):
    """
    Returns whether or not the point, given by a list [x, y], 
    is inside one of the circles with center given by centers[i]
    and radius given by radii[i] for some index, i. 
    """
    for i, center in enumerate(centers):
        dist_from_center = np.sqrt((point[0] - center[0])**2 + (point[1] - center[1])**2)
        if dist_from_center < radii[i]:
            return True
    return False
#Adding the time axis to our data
dat = {'time' : time}

#Creating an empty pixel array for each timestamp
clean_signal = np.zeros((len(time), rows,cols))

#Defining the location and sizes of each circle. 
centers = np.random.randint(100, size = (20, 2))
radii = np.random.normal(5, 1, 20)

#Populating the empty signal array
for x in tqdm(range(rows), position = 0, leave = True):
    for y in range(cols):      
        if inside(centers, radii, [x, y]):
            clean_signal[:, x, y] = square(2 * np.pi * 100 * time) + 1
        else:
            clean_signal[:, x, y] = 0.1 * square(2 * np.pi * 100 * time) + 0.1
dat['signal'] = clean_signal

Now, let's visualize a slowed down version of our generated signal, this might take a couple minutes. The animation can be finicky at times, if it doesn't work just skip this step. 

In [ ]:
%%capture
%matplotlib inline

#Scale to slow down animation by
slow_down = 25
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( 0, rows))
ax.set_ylim((0, cols))
im = ax.imshow(np.zeros((rows, cols)))

# initialization function: plot the background of each frame
def init():
    im.set_data(np.zeros((rows, cols)))
    return (im,)

# animation function. This is called sequentially
def animate(i):
    ax.imshow(clean_signal[i])
    im.set_data(clean_signal[i])
    
    return (im,)
# call the animator.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=150, interval=1000/sampling_rate * slow_down, blit = True)

 If it doesn't work, try replacing 'jshtml' on the line below with 'html5'. To use 'html5' make sure you have installed FFmpeg. 

In [ ]:
rc('animation', html='jshtml') #Maybe replace 'jshtml' with 'html5'
anim

If the animation did not work, we can just visualize the two fluorescent and non-fluorescent frames below. 

In [ ]:
fig1, ax1 = plt.subplots()
#Displaying when the sample is fluorescing
ax1.set_title("Fluorescing")
ax1.imshow(clean_signal[0])

fig12, ax2 = plt.subplots()
#Displaying when the sample is not fluorescing
ax2.set_title("Not Fluorescing")
ax2.imshow(clean_signal[sampling_rate//(2 * freq), :, :])

Now, let's add some Gaussian noise to our data

In [ ]:
mean = 0
standard_deviation = 1
noisy_signal = np.random.normal(mean, standard_deviation, clean_signal.shape) + clean_signal
dat['signal'] = noisy_signal

Visualizing our Noisy signal the same way as above, 

In [ ]:
%%capture
%matplotlib inline

#Scale to slow down animation by
slow_down = 25
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( 0, rows))
ax.set_ylim((0, cols))
im = ax.imshow(np.zeros((rows, cols)))

# initialization function: plot the background of each frame
def init():
    im.set_data(np.zeros((rows, cols)))
    return (im,)

# animation function. This is called sequentially
def animate(i):
    ax.imshow(noisy_signal[i])
    im.set_data(noisy_signal[i])
    
    return (im,)
# call the animator.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=150, interval=1000/sampling_rate * slow_down, blit = True)



In [ ]:
rc('animation', html='jshtml') #Maybe replace 'jshtml' with 'html5'
anim

And if the animation doesn't work, we can visualize it frame by frame, like before. 

In [ ]:
fig1, ax1 = plt.subplots()
#Displaying when the sample is fluorescing
ax1.set_title("Fluorescing")
ax1.imshow(noisy_signal[0])

fig12, ax2 = plt.subplots()
#Displaying when the sample is not fluorescing
ax2.set_title("Not Fluorescing")
ax2.imshow(noisy_signal[sampling_rate//(2 * freq), :, :])

Clearly, with all this noise, it is extremely difficult to figure out the relative concentrations of the relative fluoresence between the circles and the background. The lock-in is necessary to extract the signal. Now, we can create our lock-in amplifier and lock into our signal. 

In [ ]:
LIA = SILIA.Amplifier(0)

Notice that our input signal array is now 3D. SILIA is designed to handle n-dimsnsional input data as long as the first axis is the time axis. To get errorbars, we choose a split for our data into 4 windows where each window contains a third of the data. There is an error overestimation if there is no overlap between the windows and a significant number of windows, and vice versa if there is significant overlap between the windows and a fewer number of windows, so the window size and number of windows should be well balanced. This step might take a few minutes since we are running the lock-in repeatedly on large amounts of data.

In [ ]:
out = LIA.amplify(references, dat)

Now, let's visualize our output magnitudes. 

In [ ]:
mags = np.asarray(out['reference 1']['magnitudes'])
plt.imshow(mags)

The lock-in reduced noise! Now, let's check if we can correctly derive the fluoresence ratio from our result. By averaging the output magnitudes from inside the circles vs outside. Note that the exact magnitudes of the output might be scaled since the lock-in only extracts the primary Fourier component of the signal.

In [ ]:
def get_ratio(image_array):
    background_sum = 0
    num_background_points = 0
    circle_sum = 0
    num_circle_points = 0
    for x in range(len(image_array)):
        for y in range(len(image_array[0])):
            val = mags[x][y]
            if inside(centers, radii, [x, y]):
                num_circle_points += 1
                circle_sum = circle_sum + val
            else:
                num_background_points += 1
                background_sum = background_sum + val
    circle_avg = circle_sum/num_circle_points
    background_avg = background_sum/num_background_points
    return circle_avg/background_avg

ratio = get_ratio(mags)
print("The fluoresence ratio between the foreground and background after lock-in is, " + str(ratio))

This is near our expected result of 10:1!